In [1]:
import findspark
findspark.init()
import pyspark as ps
import warnings
from pyspark.sql import SQLContext

#sc = ps.SparkContext(appName='Lee')#No necesita crear contexto porque desde que arranca tiene uno.

#Esto es para leer el archivo pero no lo manipula como DataFrame
file = sc.textFile("C:/Spark/csv1.csv")
words = file.flatMap(lambda line: line.split(","))
words.count()

39

In [2]:
sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('C:/Spark/csv1.csv')
print(type(df))
print(df.show(5))

<class 'pyspark.sql.dataframe.DataFrame'>
+---+----------------+------------+
| id|          nombre|departamento|
+---+----------------+------------+
|  1|      Juan Perez|      ventas|
|  2| Josefina Orozco|      ventas|
|  3|   Jose Gonzalez|      ventas|
|  4|Olivia Hernandez|      ventas|
|  5|     Omar Garcia|  produccion|
+---+----------------+------------+
only showing top 5 rows

None


In [3]:
print(df.count())
df.printSchema()

12
root
 |-- id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- departamento: string (nullable = true)



In [6]:
print(df.select('departamento').distinct().count())
df.groupby('departamento').count().show()

3
+------------+-----+
|departamento|count|
+------------+-----+
|      ventas|    4|
|   direccion|    4|
|  produccion|    4|
+------------+-----+



In [8]:
gp=df.groupby('departamento').count()
print("Serie 1 de archivos: " + str(gp.rdd.getNumPartitions()))
gp.write.csv('depto1')
gp = gp.repartition(1)
print("Serie 2 de archivos:" + str(gp.rdd.getNumPartitions()))
gp.write.csv('depto2')

Serie 1 de archivos: 200
Serie 2 de archivos:1


In [11]:
salida = sc.textFile('C:\Spark\depto2\part-00000-846e7bd5-7b62-466b-b125-e6112704c747-c000.csv')
sa = sqlContext.read.format('com.databricks.spark.csv').options(header='false', inferschema='true').load('C:\Spark\depto2\part-00000-846e7bd5-7b62-466b-b125-e6112704c747-c000.csv')
print(sa.show(5))
#sc.stop() #Para terminar el programa.

+----------+---+
|       _c0|_c1|
+----------+---+
|    ventas|  4|
| direccion|  4|
|produccion|  4|
+----------+---+

None
